In [1]:
from pyspark.sql.functions import col, mean
from utils import load_data

In [2]:
# Cargar los datos
df = load_data()

./data/datasetMta33featues.csv ya existe.
./data/datasetLeg33featues.csv ya existe.


24/10/06 21:58:38 WARN Utils: Your hostname, andres-b460mds3h resolves to a loopback address: 127.0.1.1; using 192.168.1.72 instead (on interface enp3s0)
24/10/06 21:58:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/06 21:58:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/06 21:58:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [10]:
df.show(5)

+--------------------+-------------------+--------------------+-------------------+--------------------+------------------+-------------------+--------------------+-------------------+
|              MaxLen|             MinLen|            Len_Diff|             AvgIAT|              MinIAT|         IAT_Ratio|            FlowLEN|           FlowLENrx|          Flow_Diff|
+--------------------+-------------------+--------------------+-------------------+--------------------+------------------+-------------------+--------------------+-------------------+
|-0.00990070472439...|-1.0286089015799738|  1.0187081968555824| -0.930950849978836|-0.07967209683884144|11.684779074685812|0.47951520842275247| 0.28442779379377325|0.19508741462897922|
|  0.7226692664219299| 0.6286402445533572| 0.09402902186857265|-1.0278752447100576|-0.07974510680190673| 12.88950865992797| 1.3018399866808683|-0.03662891295780...| 1.3384688996386764|
|-0.17243597434077942| 0.6286402445533572| -0.8010762188941366|-0.858723675

In [3]:
# Observaciones donde el tráfico es anómalo
anomalous_data = df.filter(col('label') == 1)
anomalous_data.show(5)

24/10/06 21:58:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------+--------------------+-------------------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-------------------+--------------------+-------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+--------------------+------------------+--------------------+------------------+-----+
|        FinFlagDist|         SynFlagDist|        RstFlagDist|         PshFlagDist|        AckFlagDist|          DNSoverIP|          TCPoverIP|           UDPoverIP|              MaxLen|             MinLen|           StdDevLen|             AvgLen|              MaxIAT|              MinIAT|             AvgIAT|         AvgWi

In [4]:
df.describe().show()

+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|summary|         FinFlagDist|         SynFlagDist|         RstFlagDist|         PshFlagDist|         AckFlagDist|           DNSoverIP|           TCPoverIP|           UDPoverIP|              MaxLen|              MinLen|           StdDevLen|              AvgLen|              Max

In [5]:
# Agrupar por etiqueta y calcular estadísticas para cada grupo
df.groupBy("label").agg(
    mean("MaxLen").alias("Promedio MaxLen"),
    mean("AvgLen").alias("Promedio AvgLen"),
    mean("MinLen").alias("Promedio MinLen"),
    mean("MaxIAT").alias("Promedio MaxIAT"),
    mean("AvgIAT").alias("Promedio AvgIAT"),
    mean("MinIAT").alias("Promedio MinIAT")
).show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|label|     Promedio MaxLen|     Promedio AvgLen|     Promedio MinLen|     Promedio MaxIAT|     Promedio AvgIAT|     Promedio MinIAT|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  1.0|-0.32853293005371176|-0.24784876474784265| 0.22598538782865102|-0.32058113478285777|-0.36935889157937973| 0.03679004728195187|
|  0.0|  0.3736062775836766| 0.28185258137141234|-0.25698957470952066| 0.36456352917253804|  0.4200333907253716|-0.04183748829954543|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [6]:
# Diferencia entre las columnas 'MaxLen' y 'MinLen'
df = (df
    # Diferencia entre la longitud máxima y mínima de paquetes
    .withColumn('Len_Diff', col('MaxLen') - col('MinLen'))
    # Razón entre el tiempo promedio de llegada entre paquetes y el mínimo
    .withColumn('IAT_Ratio', col('AvgIAT') / col('MinIAT'))
    # Diferencia entre el flujo total y el recibido
    .withColumn('Flow_Diff', col('FlowLEN') - col('FlowLENrx'))
    .select('MaxLen', 'MinLen', 'Len_Diff', 'AvgIAT', 'MinIAT', 'IAT_Ratio',
    'FlowLEN', 'FlowLENrx', 'Flow_Diff'))

# Mostrar las nuevas columnas
df.show(5)

+--------------------+-------------------+--------------------+-------------------+--------------------+------------------+-------------------+--------------------+-------------------+
|              MaxLen|             MinLen|            Len_Diff|             AvgIAT|              MinIAT|         IAT_Ratio|            FlowLEN|           FlowLENrx|          Flow_Diff|
+--------------------+-------------------+--------------------+-------------------+--------------------+------------------+-------------------+--------------------+-------------------+
|-0.00990070472439...|-1.0286089015799738|  1.0187081968555824| -0.930950849978836|-0.07967209683884144|11.684779074685812|0.47951520842275247| 0.28442779379377325|0.19508741462897922|
|  0.7226692664219299| 0.6286402445533572| 0.09402902186857265|-1.0278752447100576|-0.07974510680190673| 12.88950865992797| 1.3018399866808683|-0.03662891295780...| 1.3384688996386764|
|-0.17243597434077942| 0.6286402445533572| -0.8010762188941366|-0.858723675

In [7]:
# Mostrar sólo registros con una diferencia de longitudes mayor a 1
df.filter(col('Len_Diff') > 1).show(5)

+--------------------+-------------------+------------------+-------------------+--------------------+------------------+-------------------+--------------------+-------------------+
|              MaxLen|             MinLen|          Len_Diff|             AvgIAT|              MinIAT|         IAT_Ratio|            FlowLEN|           FlowLENrx|          Flow_Diff|
+--------------------+-------------------+------------------+-------------------+--------------------+------------------+-------------------+--------------------+-------------------+
|-0.00990070472439...|-1.0286089015799738|1.0187081968555824| -0.930950849978836|-0.07967209683884144|11.684779074685812|0.47951520842275247| 0.28442779379377325|0.19508741462897922|
|   2.303553805344929| 0.6286402445533572|1.6749135607915717|-1.0980701129478383|-0.07849750852364294|13.988598282926478| 0.1008240158826326| -0.0494719682035481| 0.1502959840861807|
|  0.2350945268015936|-1.0286089015799738|1.2637034283815674|-0.6113041978141949| -0.

In [8]:
# Filtrar los flujos donde la razón entre llegadas de paquetes es mayor a un
# umbral
df.filter(col("IAT_Ratio") > 10).show(5)

+--------------------+-------------------+-------------------+-------------------+--------------------+------------------+-------------------+--------------------+-------------------+
|              MaxLen|             MinLen|           Len_Diff|             AvgIAT|              MinIAT|         IAT_Ratio|            FlowLEN|           FlowLENrx|          Flow_Diff|
+--------------------+-------------------+-------------------+-------------------+--------------------+------------------+-------------------+--------------------+-------------------+
|-0.00990070472439...|-1.0286089015799738| 1.0187081968555824| -0.930950849978836|-0.07967209683884144|11.684779074685812|0.47951520842275247| 0.28442779379377325|0.19508741462897922|
|  0.7226692664219299| 0.6286402445533572|0.09402902186857265|-1.0278752447100576|-0.07974510680190673| 12.88950865992797| 1.3018399866808683|-0.03662891295780...| 1.3384688996386764|
|-0.17243597434077942| 0.6286402445533572|-0.8010762188941366|-0.858723675320616

In [9]:
# Filtrar los flujos donde la razón entre llegas máxima es mayor a un umbral
df.filter(col("MaxIAT") > 1).show(5)

+------------------+------------------+-------------------+--------------------+--------------------+-------------------+------------------+--------------------+--------------------+
|            MaxLen|            MinLen|           Len_Diff|              AvgIAT|              MinIAT|          IAT_Ratio|           FlowLEN|           FlowLENrx|           Flow_Diff|
+------------------+------------------+-------------------+--------------------+--------------------+-------------------+------------------+--------------------+--------------------+
|0.9962942343041483|0.6286402445533572| 0.3676539897507911| -0.8302488724399009|-0.07974330401830021| 10.411518342021141|1.5615293651986377|  1.7693154154462842|-0.20778605024764651|
|0.7226692664219299|0.6286402445533572|0.09402902186857265|-0.16864259226085798|-0.07973879707444975|  2.114937752364152|1.1560858927809687|  -0.040542549422358|  1.1966284422033266|
|0.7226692664219299|0.6286402445533572|0.09402902186857265|  0.5305651187704029|-0.07

24/10/06 21:58:49 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
